# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
sys.path.insert(1, '../python/')
sys.path.insert(2, './python/')

import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load drawingTools.py


In [3]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v103t'
version_V8 = 'v119'

version_V9 = 'v121'

# -------------------------------------------------------------------------

samples_ele = [
#     Sample('ele_flat2to100_PU0_eg', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
#     Sample('ele_flat2to100_PU0_eg', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V8, 'hybrid')
    ]


samples_ele_calib = [
    Sample('ele_flat2to100_PU0_tps', 'PU0', 'v120', 'calib'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', 'v120', 'calib')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_eg', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V9, 'ele-V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version, 'photon')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version_V9, 'photon-V9'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version_V9, 'photon-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0_tracks', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200_tracks', 'PU200', version, 'mb')
    ]




In [4]:
do_eff = True

In [5]:
import import_ipynb
import configuration

importing Jupyter notebook from configuration.ipynb
gen_part_selections: 14
gen_part_selections: 14


In [6]:
# %load settings.py
# === samples =====================================================
# import python.selections as selections
# import python.collections as collections

import pprint


samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
# samples += samples_ele_V8
samples += samples_ele_V9
# samples += samples_ele_calib

for smp in samples:
    smp.open_file()


sample = 'ele-V9'

do_rate = False
do_eff = True

# === TP ==========================================================
tps = [
       'L1Trk',
       'L1TrkEmu',    
       'EG',
       'EGBRL',
       'EGALL',
       'TkEle',
       'TkEleEL',
#        'TkEleBRL',
       'TkEleELBRL',
#        'TkEleALL',
       'TkEleELALL'
      ]

# tps = ['HMvDRshapeDrCalib',
#        'HMvDREmInt',
#        'EG'
#       ]



# === Load the Histo Primitives ====================================
histo_primitives = pd.DataFrame()

if do_rate:
#     from python.plotters import rate_plotters, eg_rate_plotters

    for plotter in configuration.rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in configuration.eg_rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
if do_eff:
#     from python.plotters import eg_genmatched_plotters, track_genmatched_plotters, tp_genmatched_plotters
    for plotter in configuration.tp_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in configuration.eg_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in configuration.track_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
        

# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.data == tp].data_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.data == tp].gen_sel.unique().tolist()

#  ==== labels ===============================================
tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


# print selections.eg_rate_selections


--- TPs: 
[   'L1Trk',
    'L1TrkEmu',
    'EG',
    'EGBRL',
    'EGALL',
    'TkEle',
    'TkEleEL',
    'TkEleELBRL',
    'TkEleELALL']
--- TP selections:
{   'EG': [   'EGq2',
              'EGq2Pt10',
              'EGq2Pt15',
              'EGq2Pt20',
              'EGq2Pt25',
              'EGq2Pt30',
              'EGq2Pt40',
              'EGq3',
              'EGq3Pt10',
              'EGq3Pt15',
              'EGq3Pt20',
              'EGq3Pt25',
              'EGq3Pt30',
              'EGq3Pt40',
              'EGq4',
              'EGq4Pt10',
              'EGq4Pt15',
              'EGq4Pt20',
              'EGq4Pt25',
              'EGq4Pt30',
              'EGq4Pt40',
              'EGq5',
              'EGq5Pt10',
              'EGq5Pt15',
              'EGq5Pt20',
              'EGq5Pt25',
              'EGq5Pt30',
              'EGq5Pt40'],
    'EGALL': [   'all',
                 'LooseTkID',
                 'Pt10',
                 'Pt10LooseTkID',
                

/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel_launcher.py:82: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [7]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print histo_primitives[histo_primitives.data == tp].gen_sel.unique()

TP: L1Trk
['all' 'Pt2' 'Pt5' 'Pt10' 'St4' 'St4Pt2' 'St4Pt5' 'St4Pt10']
['GEN' 'GENPt15' 'GENPt10to25' 'GENPt30' 'GENPt35' 'GENPt40' 'GENEtaD'
 'GENEtaDPt15' 'GENEtaDE' 'GENEtaDEPt15' 'GENEtaBC' 'GENEtaBCPt15'
 'GENEtaBCD' 'GENEtaBCDPt15']
TP: L1TrkEmu
['all' 'Pt2' 'Pt5' 'Pt10' 'St4' 'St4Pt2' 'St4Pt5' 'St4Pt10']
['GEN' 'GENPt15' 'GENPt10to25' 'GENPt30' 'GENPt35' 'GENPt40' 'GENEtaD'
 'GENEtaDPt15' 'GENEtaDE' 'GENEtaDEPt15' 'GENEtaBC' 'GENEtaBCPt15'
 'GENEtaBCD' 'GENEtaBCDPt15']
TP: EG
['EGq2' 'EGq2Pt10' 'EGq2Pt15' 'EGq2Pt20' 'EGq2Pt25' 'EGq2Pt30' 'EGq2Pt40'
 'EGq3' 'EGq3Pt10' 'EGq3Pt15' 'EGq3Pt20' 'EGq3Pt25' 'EGq3Pt30' 'EGq3Pt40'
 'EGq4' 'EGq4Pt10' 'EGq4Pt15' 'EGq4Pt20' 'EGq4Pt25' 'EGq4Pt30' 'EGq4Pt40'
 'EGq5' 'EGq5Pt10' 'EGq5Pt15' 'EGq5Pt20' 'EGq5Pt25' 'EGq5Pt30' 'EGq5Pt40']
['GEN' 'GENPt15' 'GENPt10to25' 'GENPt30' 'GENPt35' 'GENPt40' 'GENEtaD'
 'GENEtaDPt15' 'GENEtaDE' 'GENEtaDEPt15' 'GENEtaBC' 'GENEtaBCPt15'
 'GENEtaBCD' 'GENEtaBCDPt15']
TP: EGBRL
['all' 'LooseTkID' 'Pt10' 'Pt10LooseT

In [8]:
tp_labels

{'EG': 'EG',
 'EGALL': 'EG all',
 'EGBRL': 'EG barrel',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRCalib': 'HM calib.',
 'HMvDREmInt': 'HM+dR(layer) Cl3d EM Int',
 'HMvDRshapeDrCalib': 'HM #Delta#rho < 0.015 calib.',
 'L1Trk': 'L1Track',
 'L1TrkEmu': 'L1Track EMU',
 'TkEle': 'TkEle',
 'TkEleEL': 'TkEle Ell. match',
 'TkEleELALL': 'TkEle Ell. match all',
 'TkEleELBRL': 'TkEle Ell. match barrel'}

In [9]:
tp_selection_labels

{'EGq2': 'hwQual 2',
 'EGq2Iso0p1': 'hwQual 2, Iso0p1',
 'EGq2Iso0p2': 'hwQual 2, Iso0p2',
 'EGq2Iso0p3': 'hwQual 2, Iso0p3',
 'EGq2Pt10': 'hwQual 2, p_{T}^{L1}>=10GeV',
 'EGq2Pt15': 'hwQual 2, p_{T}^{L1}>=15GeV',
 'EGq2Pt20': 'hwQual 2, p_{T}^{L1}>=20GeV',
 'EGq2Pt25': 'hwQual 2, p_{T}^{L1}>=25GeV',
 'EGq2Pt30': 'hwQual 2, p_{T}^{L1}>=30GeV',
 'EGq2Pt40': 'hwQual 2, p_{T}^{L1}>=40GeV',
 'EGq3': 'hwQual 3',
 'EGq3Iso0p1': 'hwQual 3, Iso0p1',
 'EGq3Iso0p2': 'hwQual 3, Iso0p2',
 'EGq3Iso0p3': 'hwQual 3, Iso0p3',
 'EGq3Pt10': 'hwQual 3, p_{T}^{L1}>=10GeV',
 'EGq3Pt15': 'hwQual 3, p_{T}^{L1}>=15GeV',
 'EGq3Pt20': 'hwQual 3, p_{T}^{L1}>=20GeV',
 'EGq3Pt25': 'hwQual 3, p_{T}^{L1}>=25GeV',
 'EGq3Pt30': 'hwQual 3, p_{T}^{L1}>=30GeV',
 'EGq3Pt40': 'hwQual 3, p_{T}^{L1}>=40GeV',
 'EGq4': 'hwQual 4',
 'EGq4Iso0p1': 'hwQual 4, Iso0p1',
 'EGq4Iso0p2': 'hwQual 4, Iso0p2',
 'EGq4Iso0p3': 'hwQual 4, Iso0p3',
 'EGq4Pt10': 'hwQual 4, p_{T}^{L1}>=10GeV',
 'EGq4Pt15': 'hwQual 4, p_{T}^{L1}>=15GeV',
 'EGq4

In [10]:
gen_selection_labels

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDPt15': '1.52 < |#eta^{GEN}| <= 2.8, p_{T}^{GEN}>=15GeV',
 'GENEtaBCPt15': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=15GeV',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDE': '2.4 < |#eta^{GEN}| <= 3.0',
 'GENEtaDEPt15': '2.4 < |#eta^{GEN}| <= 3.0, p_{T}^{GEN}>=15GeV',
 'GENEtaDPt15': '2.4 < |#eta^{GEN}| <= 2.8, p_{T}^{GEN}>=15GeV',
 'GENEtaF': '|#eta^{GEN}| <= 1.479',
 'GENPt10to25': '10 #leq p_{T}^{GEN} < 25GeV',
 'GENPt15': 'p_{T}^{GEN}>=15GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt35': 'p_{T}^{GEN}>=35GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV'}

In [11]:
%%time

hplot = HPlot(samples, tp_labels, tp_selection_labels, gen_selection_labels)

for tp in tps:
    hplot.cache_histo(classtype=histos.HistoSetEff, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_select[tp])



CPU times: user 14.9 s, sys: 768 ms, total: 15.7 s
Wall time: 15.6 s


In [12]:
print dir(samples[0])
samples[0].histo_file.ls()
samples

['__doc__', '__init__', '__module__', '__repr__', 'histo_file', 'histo_filename', 'label', 'name', 'open_file', 'type']


[<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v121.root, ele-V9>,
 <Sample ../plots1/histos_ele_flat2to100_PU200_eg_v121.root, ele-V9>]

TFile**		../plots1/histos_ele_flat2to100_PU0_eg_v121.root	
 TFile*		../plots1/histos_ele_flat2to100_PU0_eg_v121.root	
  KEY: TDirectoryFile	EGHistos;1	EGHistos
  KEY: TDirectoryFile	EGResoHistos;1	EGResoHistos
  KEY: TDirectoryFile	GenParticleHistos;1	GenParticleHistos


In [13]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'TkEle', 'EGq4Pt20', ['GENEtaBC'], debug=False)
print hsets
hsets[0].name
hsets[0].name
dir(hsets[0].h_eff)

[<python.l1THistos.HistoSetEff instance at 0x11a27acf8>]


['__doc__', '__init__', '__module__']

In [14]:
%%time

for smp in hplot.data['sample'].unique():
    print 'Sample: {}'.format(smp)
    for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
        for tp in tps:
#         for tp in ['EG', 'TkEleEL']:
            print 'PU: {}, TP: {}'.format(pu, tp)
            for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                if 'Pt' not in tp_sel:
                    continue
#                 tp_sel_den = 'all'
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '  {} den -> {}'.format(tp_sel, tp_sel_den) 
                for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                        continue
#                     print gen_sel
                    hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                    hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                    hset[0][0].computeTurnOn(hsetden[0][0].h_num)


Sample: ele-V9
PU: PU0, TP: L1Trk
  Pt2 den -> all
  Pt5 den -> all
  Pt10 den -> all
  St4Pt2 den -> St4
  St4Pt5 den -> St4
  St4Pt10 den -> St4
PU: PU0, TP: L1TrkEmu
  Pt2 den -> all
  Pt5 den -> all
  Pt10 den -> all
  St4Pt2 den -> St4
  St4Pt5 den -> St4
  St4Pt10 den -> St4
PU: PU0, TP: EG
  EGq2Pt10 den -> EGq2
  EGq2Pt15 den -> EGq2
  EGq2Pt20 den -> EGq2
  EGq2Pt25 den -> EGq2
  EGq2Pt30 den -> EGq2
  EGq2Pt40 den -> EGq2
  EGq3Pt10 den -> EGq3
  EGq3Pt15 den -> EGq3
  EGq3Pt20 den -> EGq3
  EGq3Pt25 den -> EGq3
  EGq3Pt30 den -> EGq3
  EGq3Pt40 den -> EGq3
  EGq4Pt10 den -> EGq4
  EGq4Pt15 den -> EGq4
  EGq4Pt20 den -> EGq4
  EGq4Pt25 den -> EGq4
  EGq4Pt30 den -> EGq4
  EGq4Pt40 den -> EGq4
  EGq5Pt10 den -> EGq5
  EGq5Pt15 den -> EGq5
  EGq5Pt20 den -> EGq5
  EGq5Pt25 den -> EGq5
  EGq5Pt30 den -> EGq5
  EGq5Pt40 den -> EGq5
PU: PU0, TP: EGBRL
  Pt10 den -> all
  Pt10LooseTkID den -> all
  Pt15 den -> all
  Pt15LooseTkID den -> all
  Pt20 den -> all
  Pt20LooseTkID den -> 

In [15]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele-V9,PU0,L1Trk,all,GEN,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a291248>
1,ele-V9,PU0,L1Trk,all,GENPt15,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a3806c8>
2,ele-V9,PU0,L1Trk,all,GENPt10to25,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a2919e0>
3,ele-V9,PU0,L1Trk,all,GENPt30,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x105589cf8>
4,ele-V9,PU0,L1Trk,all,GENPt35,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a3801b8>
5,ele-V9,PU0,L1Trk,all,GENPt40,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a36aa28>
6,ele-V9,PU0,L1Trk,all,GENEtaD,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a291200>
7,ele-V9,PU0,L1Trk,all,GENEtaDPt15,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x1190e8b00>
8,ele-V9,PU0,L1Trk,all,GENEtaDE,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a36ae60>
9,ele-V9,PU0,L1Trk,all,GENEtaDEPt15,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11a291050>


In [16]:
hplot.data['sample'].unique()

array(['ele-V9'], dtype=object)

In [17]:
hplot.data['tp'].unique()

array(['L1Trk', 'L1TrkEmu', 'EG', 'EGBRL', 'EGALL', 'TkEle', 'TkEleEL',
       'TkEleELBRL', 'TkEleELALL'], dtype=object)

In [18]:
hplot.data[hplot.data.tp == 'L1TrkEmu'].tp_sel.unique()

array(['all', 'Pt2', 'Pt5', 'Pt10', 'St4', 'St4Pt2', 'St4Pt5', 'St4Pt10'], dtype=object)

In [19]:
hplot.data[hplot.data.tp == 'TkEle'].tp_sel.unique()

array(['EGq2', 'EGq2Pt10', 'EGq2Pt15', 'EGq2Pt20', 'EGq2Pt25', 'EGq2Pt30',
       'EGq2Pt40', 'EGq3', 'EGq3Pt10', 'EGq3Pt15', 'EGq3Pt20', 'EGq3Pt25',
       'EGq3Pt30', 'EGq3Pt40', 'EGq4', 'EGq4Pt10', 'EGq4Pt15', 'EGq4Pt20',
       'EGq4Pt25', 'EGq4Pt30', 'EGq4Pt40', 'EGq5', 'EGq5Pt10', 'EGq5Pt15',
       'EGq5Pt20', 'EGq5Pt25', 'EGq5Pt30', 'EGq5Pt40', 'EGq2Iso0p2',
       'EGq2Iso0p1', 'EGq2Iso0p3', 'EGq3Iso0p2', 'EGq3Iso0p1',
       'EGq3Iso0p3', 'EGq4Iso0p2', 'EGq4Iso0p1', 'EGq4Iso0p3',
       'EGq5Iso0p2', 'EGq5Iso0p1', 'EGq5Iso0p3'], dtype=object)

In [20]:
# sample='ele'

In [21]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', ['GENEtaBC'], debug=False)
print hsets
hsets[0].name
print dir(hsets[0].h_eff)


[<python.l1THistos.HistoSetEff instance at 0x11aae8ab8>]
['__doc__', '__init__', '__module__', 'h_abseta', 'h_energy', 'h_eta', 'h_fBrem', 'h_pt', 'h_reachedEE']


## Effinciency vs $\eta$

In [22]:
dir(hplot)


['__doc__',
 '__init__',
 '__module__',
 'cache_histo',
 'data',
 'get_histo',
 'labels_dict',
 'pus',
 'tp_sets']

In [23]:
# newconfig=tdr_config
# newconfig.legend_size=(0.5, 0.2)
# newconfig.legend_position=(0.4, 0.2)

# dm = DrawMachine(newconfig)
# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-V9'], ['PU200'], ['EG'], 'EGq5Pt25', 'GENPt30', debug=False)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG calo-only'])

# # hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid'], ['PU200'], ['L1Trk', ], ['Pt10'], 'GENPt30', debug=False)
# # dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['L1 tracks (p_{T} > 10 GeV)'])
# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-V9'], ['PU200'], ['TkEle', 'TkEleEL'], 'EGq5Pt25', 'GENPt30', debug=False)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG track-match', 'EG track-match (Elliptic cut)'])


# dm.draw(text='EG endcap, p_{T}^{GEN}>25GeV', x_min=1.5, x_max=3.2, y_min=0.5, y_max=1.1, v_lines=[1.52, 2.4])
# dm.write(name='egEndcap_matchingEffBreakdown')

In [24]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele-V9'], ['PU200'], ['TkEleEL'], 'EGq5Pt10', 'GENPt15', debug=False)
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [25]:
# for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GEN']:
#             hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], tp, tp_sel, gen_sel, debug=False)
#             # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
#             draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
# #             print [his.h_num.h_eta.GetName() for his in hsets]

In [26]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleEL', 'EGq5Pt10', 'GENPt15', debug=False)
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [27]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleELALL', 'Pt10LooseTkID', 'GENPt15', debug=False)

draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])

In [28]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleEL', 'EGq5Pt25', 'GENPt35', debug=False)
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [29]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleELALL', 'Pt25LooseTkID', 'GENPt35', debug=False)
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[1.52, 1.7, 2.4, 2.8])


In [30]:
# sample='hybrid'

In [31]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU200'], ['EGALL', 'TkEleELALL'], ['Pt25LooseTkID', 'Pt25Iso0p1'], ['GENPt35'], debug=False)

for hset in hsets:
    hset.computeEff(rebin=2)

# labels = ['TP head', 'L1 menu version', 'backported']
text_forprint = 'EG, p_{T}^{L1} #geq 25GeV, p_{T}^{GEN} #geq 35GeV'


labels_final = []
labels_final.append('calorimeter-only')
labels_final.append('track-matched')
labels_final.append('#splitline{track-matched + charged iso.}{(no EG ID for |#eta| < 1.479)}')


canv = draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels_final, text=text_forprint, 
     min_x=0, max_x=3.5, min_y=0.6, max_y=1.1, v_lines=[1.47, 1.52, 2.4],
     y_axis_label='efficiency',
    legend_position = (0.16, 0.2, 0.42, 0.35),
     do_write=True, write_name='figs/egammaBarrelEndcap_EG_GENPt30_hEffVAbsEta')

# print canv.GetByType(ROOT.TLegend)
# leg2 = getLegend(0.56, 0.15, 0.92, 0.30)
# leg2.SetHeader('|#eta|>1.479')
# for idx,his in enumerate(hsets):
#     leg2.AddEntry(his.h_eff.h_eta.CreateGraph(), labels_final[idx], 'P')
# leg2.Draw('same')
# canv.Draw()
# SaveCanvas(canv, 'test')
# stuff[-7].AddEntry(ROOT.TObject(), 'PIPPO', '')
# stuff[-7].Draw('same')
# canv.Draw()

# print canv.GetListOfPrimitives().Print()

draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels_final, text=text_forprint, 
     min_x=-3.5, max_x=3.5, min_y=0.6, max_y=1.1, v_lines=[-1.47, -1.52, -2.4, 1.47, 1.52, 2.4],
     y_axis_label='eff. (L1 matched to GEN/GEN)',
     do_write=True, write_name='figs/egammaBarrelEndcap_EG_GENPt30_hEffVEta')



ERROR in TPDF::Open: Cannot open file:figs/egammaBarrelEndcap_EG_GENPt30_hEffVAbsEta.pdf
ERROR in TPDF::Open: Cannot open file:figs/egammaBarrelEndcap_EG_GENPt30_hEffVEta.pdf


In [32]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU200'], 'EG', ['EGq5Pt25'], ['GENPt35'], debug=False)
# labels = ['TP head', 'L1 menu version', 'backported']
text_forprint = 'EG endcap, p_{T}^{L1} #geq 25GeV, p_{T}^{GEN} #geq 35GeV'
draw([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_x=1.5, max_x=3.5, min_y=0.6, max_y=1.1, v_lines=[1.52, 2.4],
     y_axis_label='eff. (L1 matched to GEN/GEN)',
     do_write=True, write_name='figs/egammaEndcap_EG_EGq5Pt25_GENPt35_hEffVEta')


ERROR in TPDF::Open: Cannot open file:figs/egammaEndcap_EG_EGq5Pt25_GENPt35_hEffVEta.pdf


In [33]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG','TkEleEL'], ['EGq5Pt25', 'EGq5Iso0p1'], 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


Info in <TCanvas::Print>: pdf file egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta.pdf has been created


In [34]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['TkEleEL'], ['EGq4Pt25', 'EGq4Iso0p2Pt25', 'EGq4Iso0p3Pt25', 'EGq5Pt25'], 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [35]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleBRL', 'TkEleELBRL'], ['Pt30'], ['GENPt40'], debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=False, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')



In [36]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleELBRL'], 'Pt20', 'GENPt30', debug=False)

newset = []

h_barrel = hsets[0].h_eff.h_eta.CreateGraph()

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleEL'], 'EGq5Pt20', 'GENPt30', debug=False)

h_endcap = hsets[0].h_eff.h_eta.CreateGraph()

newset.append(h_barrel)
newset.append(h_endcap)

newlabels = ['barrel', 'endcap']

draw(newset, newlabels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


Info in <TCanvas::Print>: pdf file egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta.pdf has been created


In [37]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleBRL', 'TkEleELBRL'], 'Pt20', 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=False, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


## Efficiency vs $p_{T}$

In [38]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [39]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleEL', 'EGq5', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [40]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleEL', 'EGq5Pt25', 'GENEtaBC', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [41]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleELALL', 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [42]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['hybrid', 'ele-V9'], ['PU200'], 'TkEleELALL', 'LooseTkID', 'GENEtaF', debug=False)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, h_lines=[0.95, 1.])


In [43]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', 'GENEtaBC', debug=False)
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
print dir(hsets[0].h_eff)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1],
     do_write=True, write_name='figs/egammaEndcap_EG_EGq5Pt25_GENEtaBC_hEffvPt')



hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5Pt25', ['GENEtaDE'])
print labels
text_forprint = 'EG endcap, 2.4 < |#eta^{GEN}| #leq 3.0, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1],
     do_write=True, write_name='figs/egammaEndcap_EG_EGq5Pt25_GENEtaDE_hEffvPt')


['__doc__', '__init__', '__module__', 'h_abseta', 'h_energy', 'h_eta', 'h_fBrem', 'h_pt', 'h_reachedEE']
['']


ERROR in TPDF::Open: Cannot open file:figs/egammaEndcap_EG_EGq5Pt25_GENEtaBC_hEffvPt.pdf
ERROR in TPDF::Open: Cannot open file:figs/egammaEndcap_EG_EGq5Pt25_GENEtaDE_hEffvPt.pdf


In [44]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', ['all', 'EGq5'], 'GENEtaBC', debug=False)
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
print dir(hsets[0].h_eff)
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1])



hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq5', ['GENEtaDE'])
print labels
text_forprint = 'EG endcap, 2.4 < |#eta^{GEN}| #leq 3.0, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0, max_y=1.1, y_axis_label='efficiency', h_lines=[0.95, 1])


['__doc__', '__init__', '__module__', 'h_abseta', 'h_energy', 'h_eta', 'h_fBrem', 'h_pt', 'h_reachedEE']
['']


In [45]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG', 'TkEleEL'], 'EGq5Pt25', 'GENEtaBC')
labels=['calorimeter only', 'track-matched']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, min_y=0, max_y=1.1, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/egammaEndcap_TkEle_EGq5Pt25_GENEtaBC_hEffvPt')


ERROR in TPDF::Open: Cannot open file:figs/egammaEndcap_TkEle_EGq5Pt25_GENEtaBC_hEffvPt.pdf


In [46]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGBRL'], ['Pt20', 'Pt20LooseTkID'], ['GENEtaF'])
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [47]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGBRL'], ['Pt25', 'Pt25LooseTkID'], ['GENEtaF', 'all'])
for hset in hsets:
    hset.computeEff(rebin=2)

labels=['w/o EG ID', 'w EG ID']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG barrel, |#eta^{GEN}| #leq 1.47, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, 
     min_y=0.8, max_y=1.02, min_x=0.0, max_x=100, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/EgammaBarrelOnlyCaloEff')


ERROR in TPDF::Open: Cannot open file:figs/EgammaBarrelOnlyCaloEff.pdf


In [48]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EGALL', 'TkEleELALL'], 'Pt25LooseTkID', 'GENEtaF')
labels=['calorimeter only', 'track-matched']
legend_position = (0.6, 0.5, 0.86, 0.6)
text_forprint='EG barrel, |#eta^{GEN}| #leq 1.479, p_{T}^{L1} #geq 25GeV'
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text_forprint, min_y=0, max_y=1.1, 
     y_axis_label='efficiency', h_lines=[0.95, 1],
     legend_position=legend_position,
     do_write=True, write_name='figs/egammaBarrel_TkEle_LooseTkIDPt25_GENEtaF_hEffvPt')


ERROR in TPDF::Open: Cannot open file:figs/egammaBarrel_TkEle_LooseTkIDPt25_GENEtaF_hEffvPt.pdf


## Efficiency vs fBrem

In [49]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk #stubs > 3")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk EMU")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


AttributeError: HistoEff instance has no attribute 'h_fBrem'

In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Turn-on curves

In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele-V9', ['PU200'], ['EG'], ['EGq4Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleEL'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


## Scaling

In [ ]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(1, 0, 50)

tf_yc.SetParLimits(3, 0.9, 1)
tf_yc.SetParLimits(0, 0, 1)
tf_yc.SetParLimits(2, 0.1, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [ ]:
sample

In [ ]:
hplot.data[(hplot.data['sample'] == sample) & (hplot.data.tp == 'TkEle')].tp_sel.unique()


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, 'PU0', 'TkEle', 'EGq3Pt15', 'GENEtaBC')


print hsets[0].name

In [ ]:
fake_histoname = uuid.uuid4().hex[:6]


In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


# tps_scaling = ['EG', 'TkEleEL']
# tp_sel_scaling = [('EGq3Pt{}', 'EGq3'), ('EGq4Pt{}', 'EGq4'), ('EGq5Pt{}', 'EGq5')]
# pt_points = [10, 15, 20, 25, 30, 40]

sample = 'ele-V9'
tps_scaling = ['EG']
tp_sel_scaling = [('EGq5Pt{}', 'EGq5')]
pt_points = [10, 15, 20, 25, 30, 40]

# sample = 'calib'
# tps_scaling = ['HMvDRshapeDrCalib']
# tp_sel_scaling = [('EmPt{}', 'Em')]
# pt_points = [10, 20, 30]


for pu in ['PU0', 'PU200']:
    for gen_sel in ['GENEtaBC']:
        for tp in tps_scaling:
            for tp_sel_strings in tp_sel_scaling:
                pt_95 = []
                pt_95_err = []
                for pt in pt_points:
#                     for (i=0; i<ngr; i++)
#       grint->SetPoint(i, gr->GetX()[i], 0);
        
                    tp_sel = tp_sel_strings[0].format(pt)
                    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, gen_sel)
                        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        #                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])

#                     if pt == 20:
                    tf_yc.SetParameters(0.6, pt, 0.1, 1.0, 0.0)
#                     else:
#                         tf_yc.SetParameters(1, 20, 4, 1.0, 0.0)
                    h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

                    result = h_eff_vs_pt[0].Fit(tf_yc, 'EMQRS+')

                    grint = ROOT.TGraphErrors(h_eff_vs_pt[0].GetN())
                    stuff.append(grint)
                    grint.SetTitle("Fitted line with .95 conf. band")
                    for point in range(0, h_eff_vs_pt[0].GetN()):
                        grint.SetPoint(point, h_eff_vs_pt[0].GetX()[point], 0)
                    ROOT.TVirtualFitter.GetFitter().GetConfidenceIntervals(grint)

                        
                    print '------- {}'.format( text)
                    print result.Print()
                    pt095 = tf_yc.GetX(0.95)
                    pt_95.append(pt095)

                    prev_err = 0
                    error = 0
                    for idx,point in enumerate(grint.GetX()):
#                         print idx, point, grint.GetEY()[idx]
                        if pt095 > point:
                            prev_err = grint.GetEY()[idx]
                        else:
                            error = (prev_err+grint.GetEY()[idx])/2
                            break
                    pt_95_err.append(error)
                    print '       pt th: {}, 95% eff: {}, err: {}'.format(pt, pt095, error)
                    #                 points.append((tp, pu, pt, pt095))
                    draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                    ROOT.gStyle.SetOptFit(11111)
                    grint.Draw('same')

                print len(pt_points)
                print len(pt_95)
                print len(pt_95_err)
                print pt_95_err
                
                graph = ROOT.TGraphErrors(len(pt_points), array.array('d', pt_points), 
                                                          array.array('d', pt_95),
                                                          array.array('d', [0.]*len(pt_points)),
                                                          array.array('d', pt_95_err) )
                graph.SetMarkerStyle(7)
                graph.SetMarkerColor(2)
                stuff.append(graph)

                hplot.data = hplot.data.append({'sample': sample,
                                                'pu': pu,
                                                'tp': tp,
                                                'tp_sel': tp_sel_strings[1],
                                                'gen_sel': gen_sel,
                                                'classtype': fake_histoname,
                                                'histo': HWrapper(graph),},
                                                ignore_index=True)

                
                

In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]


In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique()

In [ ]:
scaling_params = pd.DataFrame(columns=['tp', 'tp_sel', 'params'])
ROOT.gStyle.SetOptFit(11111)

for sample in hplot.data[hplot.data.classtype == fake_histoname]['sample'].unique():
    print sample
    for pu in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample)].pu.unique():
        for tp in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample) & (hplot.data.pu == pu)].tp.unique():
            for tp_sel in hplot.data[(hplot.data.classtype == fake_histoname) & (hplot.data['sample'] == sample) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                hsets, labels, text = hplot.get_histo(fake_histoname, sample, pu, tp, tp_sel, ['GENEtaBC'])
                result = hsets[0].Fit('pol1', 'MES+', '', 5, 35)
                result.GetParams()
                scaling_params = scaling_params.append({'pu': pu,
                                                         'tp': tp,
                                                         'tp_sel': tp_sel+'EtaBC',
                                                         'params': (result.GetParams()[0], result.GetParams()[1])},
                                                          ignore_index=True)
                draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

###### 

In [ ]:
scaling_params

In [ ]:
scaling_params.to_csv(r'scaling_params_{}.csv'.format(version_V9))

In [ ]:
eta_boundaries_fiducial = [-5, -4, -3,-2.5 ,-1.5, -0.5, 0.5, 1.5, 2.5, 3, 4, 5]
eta_boundaries = []
phi_boundaries_fiducial = []


eta_overlap = 0.25
phi_overlap = 0.25
phiSlices = 9

phiWidth = 2*math.pi/phiSlices


phi_boundaries = []


for ieta,eta_low_fiducial in enumerate(eta_boundaries_fiducial):dd
    if ieta>=len(eta_boundaries_fiducial)-1:
        break
    eta_high_fiducial = eta_boundaries_fiducial[ieta+1]
    print 'fiducial boundaries: {}, {}'.format(eta_low_fiducial, eta_high_fiducial)
    eta_low = eta_low_fiducial - eta_overlap
    eta_high = eta_high_fiducial + eta_overlap
    eta_boundaries.append((eta_low, eta_high))
    
    
    
for iphi in range(0,9):
    phiCenter = (iphi+0.5)*phiWidth-math.pi
    print iphi,phiCenter
    phi_low_fiducial = phiCenter - phiWidth/2.
    phi_high_fiducial = phiCenter+phiWidth/2.
    
    print 'fiducial boundaries: {}, {}'.format(phi_low_fiducial, phi_high_fiducial)
    phi_boundaries_fiducial.append(phi_low_fiducial)
    
    phi_low = phi_low_fiducial - phi_overlap
    if phi_low < -1*math.pi:
        phi_low = math.pi-phi_overlap
    phi_high = phi_high_fiducial + phi_overlap
    if phi_high > math.pi:
        phi_high = -1*math.pi+phi_overlap
    
    phi_boundaries.append((phi_low, phi_high))
    print 'boundaries: {} {}'.format(phi_low, phi_high)
    
    
phi_boundaries_fiducial.append(math.pi)
print "FIDUCIAL"
print eta_boundaries_fiducial
print phi_boundaries_fiducial

print "ACTUAL"
print phi_boundaries
print eta_boundaries